In [996]:
## Basic Packages

import time
from math import sqrt, pi
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re

## Sklearn packages

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE

## Spotipy packages (Spotify API)

import spotipy.oauth2
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import spotipy

## Importing Bokeh to explore outliers

from bokeh.plotting import figure, output_file, show
import bokeh.models as bmo
from bokeh.palettes import d3
from bokeh.models import Legend, BoxSelectTool, BoxZoomTool, LassoSelectTool
import bokeh.plotting as bpl
from bokeh.embed import file_html, components, autoload_static
import bokeh.embed as bem
from bokeh.resources import CDN

## Notebook display settings

pd.options.display.max_columns = 999

start_time = time.time()

## Audio feature descriptions

#https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

In [297]:
sp_username = **USERNAME**
sp_client_id = **ID**
sp_client_secret = **SECRET**
sp_redirect_uri = **URI**

SyntaxError: invalid syntax (<ipython-input-297-75d88d7113a5>, line 1)

In [1912]:
token = util.prompt_for_user_token(username = sp_username, client_id= sp_client_id,client_secret= sp_client_secret,redirect_uri=sp_redirect_uri)

In [1795]:
sp = spotipy.Spotify(auth=token)

## Functions Related to API Calls

In [1000]:
def sp_search_artist(sp_artist_name):
        ## Takes user name from user input and searches for artist information - returns dataframe
        artist_search_results = sp.search(q = sp_artist_name, limit = 1, type = 'artist')
        artist_search_results_2 = artist_search_results['artists']
        artist_search_results_3 = artist_search_results_2['items']
        sp_search_artists_df = pd.DataFrame(artist_search_results_3)
        return sp_search_artists_df

In [1001]:
def sp_search_track(sp_track_id):
        ## Takes user name from user input and searches for artist information - returns dataframe
        track_search_results = sp.search(q = sp_track_id, limit = 10, type = 'track')
        track_search_results_2 = track_search_results['track']
        track_search_results_3 = track_search_results_2['items']
        sp_search_track_df = pd.DataFrame(track_search_results_3)
        return sp_search_track_df

In [1002]:
def sp_search_album(sp_artist_id):
        ## Search for albums based on artist ID
        album_search_results = sp.artist_albums(sp_artist_id, album_type = 'album')
        album_search_results_2 = album_search_results['items']
        sp_search_album_df = pd.DataFrame(album_search_results_2)       
        return sp_search_album_df

In [1003]:
def sp_search_track(album, alb_num, rel_date):
        ## Function that utilizes Spotify's album_tracks method to return all tracks from an album ID
        track_search_results = sp.album_tracks(album, limit = 50)
        track_search_results_2 = track_search_results['items']
        sp_search_track_df = pd.DataFrame(track_search_results_2)
        sp_search_track_df['album_name'] = alb_num
        sp_search_track_df['release_date'] = rel_date
        return sp_search_track_df

In [1004]:
def sp_features_audio(song_list):
        ## Function that utilizes Spotify's audio_features method to return audio features (energy, tempo, etc)
        audio_features_results = sp.audio_features(tracks = song_list)
        audio_df = pd.DataFrame(audio_features_results)
        return audio_df

In [1005]:
def audio_track_analysis(song_list):
        ## Function that utilizes Spotify's audio_features method to return audio features (energy, tempo, speechiness, etc)
        audio_analysis_list = []
        for i in range(len(song_list)):
            results = sp.audio_analysis(song_list[i])
            audio_analysis_list.append(results)
        track_df = pd.DataFrame(audio_analysis_list)
        return track_df

In [1006]:
def sp_track_info(song_list):
        track_info_results = sp.tracks(song_list)
        track_info_df = pd.DataFrame(track_info_results['tracks'])
        return track_info_df

## General functions (not related to Spotify API)

In [1007]:
def count_stats (column, df):
    count_list = []
    for row in range(len(df)):
        count = 0
        for item in range(len(df.iloc[row, column])):
            count += 1
        count_list.append(count)
    return count_list

In [1008]:
## Function that takes a dataframe and iterates through columns to calculate predicted value

def future_value(df, req_feat):
    ## Will iterate through columns to calculate future value
    requested_df = df[df.feature == req_feat]

    x_reg = np.array(requested_df.album_number)
    y_reg = np.array(requested_df.value)

    fit = np.polyfit(x_reg, y_reg, 1)
    fit_fn = np.poly1d(fit) 

    ## Fit the linear regression model to the expected 14th value
    return fit_fn(14)

In [1009]:
## Function to normalize features

def feature_norm(df, col):
    return(df[col] - df[col].min())/(df[col].max() - df[col].min())

## Creating Dataframes from API Calls

In [1010]:
## Search for artist by input

#artist_search_df = sp_search_artist(input("What artists would you like to see? "))
artist_search_df = sp_search_artist("A Fine Frenzy")

## Collect artist id and genres

artist_search_id = list(artist_search_df.id)

artist_search_genre = list(artist_search_df.genres)
artist_search_genre = artist_search_genre[0]

In [1011]:
## Starting to build album information, then saving off information for later calls

album_search_df = sp_search_album(artist_search_id[0])

In [1012]:
## BEATLES ONLY

# first_album_search = sp.album('3KzAvEXcqJKBF97HrXwlgf')

# first_album_search_2 = first_album_search['tracks']
# first_album_search_3 = first_album_search_2['items']
# first_album_df = pd.DataFrame(first_album_search_3)

# first_album_df['album_name'] = 'Please Please Me (Remastered)'
# first_album_df['release_date'] = '1963-03-22'

In [1013]:
## Cutting off unwanted albums from Beatles

# album_search_df = album_search_df.iloc[7:,:]

album_search_df_2 = album_search_df[album_search_df.duplicated(subset = 'release_date', keep = 'last') == False]

album_search_df_2.reset_index(drop = True, inplace = True)

album_name_list = list(album_search_df.name)
album_id = list(album_search_df.id)
album_release_date = list(album_search_df.release_date)

In [1014]:
## Iterating through albums to return a dataframe with track information

for i in range(len(album_id)):
    if i == 0:
        album_track_df = sp_search_track(album_id[i], album_name_list[i], album_release_date[i])
    else:
        new_track_df = sp_search_track(album_id[i], album_name_list[i], album_release_date[i])
        album_track_df = album_track_df.append(new_track_df)
        album_track_df.reset_index(drop = True, inplace = True)

In [1015]:
#album_track_df = album_track_df[album_track_df.album_name != "Sgt. Pepper's Lonely Hearts Club Band (Deluxe Edition)"]

In [1016]:
## BEATLES ONLY

# album_track_df = pd.concat([first_album_df, album_track_df])

In [1017]:
#album_track_df.reset_index(drop = True, inplace = True)

In [1018]:
## Track list will be used to get audio_features and track_analysis for each song        
        
track_list = list(album_track_df.id)

## Audio features for each song

for i in range(len(track_list)):
    if i == 0:
        audio_features_df = sp_features_audio(track_list[i])
    else:
        new_audio_features_df = sp_features_audio(track_list[i])
        audio_features_df = audio_features_df.append(new_audio_features_df)
        audio_features_df.reset_index(drop = True, inplace = True)  

In [1019]:
## Track analysis for each song

track_analysis_df = audio_track_analysis(track_list)
track_analysis_df.drop(columns = ['meta', 'track'], inplace = True)

## Creating counts for each track_analysis record

bars_count = count_stats(0, track_analysis_df)
beats_count = count_stats(1, track_analysis_df)
sections_count = count_stats(2, track_analysis_df)
segments_count = count_stats(3, track_analysis_df)
tatums_count = count_stats(4, track_analysis_df)

audio_df = pd.DataFrame({'bars_count': bars_count, 'beats_count': beats_count, 
                      'sections_count': sections_count, 'segments_count': segments_count, 
                      'tatums_count': tatums_count})

In [1020]:
## Appending track id numbers to audio analysis

track_series = pd.Series(track_list)

audio_df = pd.concat([audio_df,track_series], axis = 1)
audio_df = audio_df.rename(columns = {0 : 'id'})

## Combining audio features and audio analysis features

combined_df = pd.merge(album_track_df, audio_features_df, how = 'left', on = 'id')

## Renaming columns in merged_df

combined_df = combined_df[['href', 'id', 'name', 'track_number', 'album_name', 'release_date',
    'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 
    'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']]

audio_df = pd.concat([combined_df, audio_df], axis=1)

## Creating a new field for song length (in seconds)

song_length = []

for i in range(len(album_track_df)):
    song_seconds = (album_track_df['duration_ms'][i] / 1000)
    song_length.append(song_seconds)

## Calculating beats per minute and 

song_len = pd.Series(song_length)

audio_analysis = pd.concat([audio_df, song_len], axis = 1)

audio_analysis.rename(columns = {0 : 'song_length_seconds'}, inplace = True)

audio_analysis['bpm'] = (audio_analysis['beats_count'] / (audio_analysis['song_length_seconds'] / 60))

audio_analysis = audio_analysis.rename(columns = {'name' : 'song_name'})

cols_to_use = combined_df.columns.difference(audio_analysis.columns)

merged_df = pd.merge(audio_analysis, combined_df[cols_to_use], how = 'left', left_index=True, right_index=True)

## Trimming down the audio features that I want to explore initially

features_df = combined_df[['release_date', 'album_name', 'name', 'acousticness', 'danceability', 'energy', 
    'loudness', 'key', 'mode', 'tempo', 'valence']]

## Separating the release date into year, month, day columns

features_df['year'], features_df['month'], features_df['day'] = features_df['release_date'].str.split('-',2).str

## Sort oldest album to newest album

features_df = features_df.sort_values(by = ['year', 'month', 'day'])

## Reset index and assign unique release dates to a list that will be used in the next loop

features_df.reset_index(drop = True, inplace = True)

unique_release = list(features_df.release_date.unique())

## For loop that assigns an album number based on the release date for future plotting

album_number = []

count = 0
record_number = 1

for row in range(len(features_df)):
    if features_df.release_date.iloc[row] == unique_release[count]:
        #print(features_df.release_date.iloc[row], "-------->", unique_release[count], "------->", features_df.release_date.iloc[row] == unique_release[count])
        album_number.append(record_number)
    else:
        #print('Change Album')
        record_number += 1
        count += 1
        album_number.append(record_number)

## Converting resulting list into a dataframe to concat, then renaming the column to show album_number

album_number = pd.DataFrame(album_number)

features_df = pd.concat([features_df, album_number], axis = 1)

features_df.rename(columns = {0 : 'album_number', 'name' : 'song_name'}, inplace = True)

/Users/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1033]:
## A Fine Frenzy Only

features_df = features_df[features_df.duplicated(subset = ['album_name', 'song_name'], keep = 'first') == False]

features_df = features_df[features_df.duplicated(subset = ['release_date', 'song_name'], keep = 'first') == False]

features_df = features_df.iloc[:-2,:]

features_df.reset_index(drop = True, inplace = True)

## Creating Raw Dataframe for Linear Regression - Song Recommender

In [1041]:
cols_to_use_2 = merged_df.columns.difference(features_df.columns)


raw_features_df_2 = pd.merge(merged_df[cols_to_use_2], features_df, how = 'left', left_on='name', right_on='song_name')
raw_features_df = pd.merge(merged_df[cols_to_use_2], features_df, how = 'left', left_on='name', right_on='song_name')

In [1043]:
raw_features_df_2 = raw_features_df_2[pd.notnull(raw_features_df_2.acousticness)]
raw_features_df = raw_features_df[pd.notnull(raw_features_df.acousticness)]

In [1035]:
#raw_features_df.to_csv('beatles.csv')

## Creating a melted dataframe for predicting future song features

In [1044]:
## Rearranging columns

melted_df = raw_features_df[['release_date',
                            'album_number',
                            'album_name',
                            'track_number',
                            'song_name',
                            'year',
                            'month',
                            'day',
                            'acousticness',
                            'danceability',
                            'energy',
                            'loudness',
                            'valence',
                            'song_length_seconds']]

In [1045]:
## Reshaping dataframe to a long format

melted_df = pd.melt(melted_df, 
                   id_vars = ['release_date',
                            'album_number',
                            'album_name',
                            'track_number',
                            'song_name',
                            'year',
                            'month',
                            'day'], 
                   var_name = 'feature', value_name = 'value')

## Performing PCA on requested artist

In [1182]:
## Normalizing data for plotting

feature_col_names = ['acousticness',
                     'danceability',
                     'energy',
                     'loudness',
                     'bars_count',
                     'beats_count',
                     'sections_count',
                     'segments_count',
                     'tatums_count',
                     'song_length_seconds',
                     'bpm']

## Iterating through columns to normalize the data 

for i in feature_col_names:
    raw_features_df[i] = feature_norm(raw_features_df, i)

## Reducing features

pca_df = raw_features_df[['album_number',
                     'acousticness',
                     'danceability',
                     'energy',
                     'bars_count',
                     'sections_count',
                     'segments_count',
                     'tatums_count',
                     'song_length_seconds',
                     'bpm']]

## Splitting data into target and features

y = pca_df['album_number']
X_norm = pca_df.iloc[:,1:]

## Requesting to use n_compenents that accounts for 95% of variance and applying fit_transform to the model

pca = sklearnPCA(n_components = .95)
transformed = pd.DataFrame(pca.fit_transform(X_norm))

## Print statements to show the impact of each feature

pca_len = len(pca.explained_variance_)

In [1187]:
## Print statements to show the impact of each feature

print(pca.explained_variance_)

print(pd.DataFrame(pca.components_,columns=X_norm.columns))

[0.27292204 0.17060973 0.04507612 0.03773001]
   acousticness  danceability    energy  bars_count  sections_count  \
0      0.590668     -0.304043 -0.507511   -0.240850       -0.181266   
1      0.387361     -0.282862 -0.251678    0.411712        0.331008   
2     -0.151056     -0.730952  0.237901    0.119366       -0.266891   
3      0.324615      0.507827 -0.121439    0.093655       -0.112743   

   segments_count  tatums_count  song_length_seconds       bpm  
0       -0.258264     -0.262442            -0.208998 -0.176010  
1        0.328595      0.384826             0.408244  0.069262  
2       -0.191736      0.124634            -0.163504  0.470941  
3       -0.074370      0.111176            -0.203789  0.735586  


## Linear Discriminant Analysis - Requested Artist

In [1774]:
## Defining 4 components for LDA and applying fit_transform

lda = LinearDiscriminantAnalysis(n_components = pca_len)
transformed_lda = pd.DataFrame(lda.fit_transform(X_norm, y))

## Defining 4 components for LDA and applying fit_transform

tsne_req = TSNE(n_components = 2, perplexity = 60, learning_rate = 900, n_iter = 1000)
transformed_tsne = pd.DataFrame(tsne_req.fit_transform(transformed_lda))

## Creating new df for interactive Bokeh graphs

tsne_df = pd.concat([raw_features_df[['album_name', 'song_name', 'album_number']], transformed_tsne.iloc[:,:2]], axis=1, join_axes=[raw_features_df.index])

tsne_df.rename(columns = {0 : 'x', 1 : 'y'}, inplace = True)

In [1775]:
tsne_df = tsne_df[tsne_df.duplicated(subset = ['album_name', 'song_name'], keep = 'first') == False]

In [1776]:
## Beatles Only

#color_palette = ['#212F3C', '#515A5A', '#873600', '#979A9A', '#186A3B', '#117A65', 
#                 '#17A589', '#2874A6', '#5B2C6F', '#884EA0', '#78281F', '#A93226', '#CB4335', '#141414']
                 

In [1777]:
## Beatles Only

#album_name_list = ['Please Please Me (Remastered)', 'With The Beatles (Remastered)', "A Hard Day's Night (Remastered)",
#                        'Beatles For Sale (Remastered)', 'Help! (Remastered)', 'Rubber Soul (Remastered)', 'Revolver (Remastered)',
#                        "Sgt. Pepper's Lonely Hearts Club Band (Remastered)", 'Magical Mystery Tour (Remastered)',
#                        'The Beatles (Remastered)', 'Yellow Submarine (Remastered)', 'Abbey Road (Remastered)', 'Let It Be (Remastered)']

In [1778]:
## AFF Only

# album_name_list = ['PINES', 'Bomb In A Birdcage', 'One Cell In The Sea']

# palette_1 = palette[11]

# palette_2 = ['#000000', '#525b91', '#4e6b4e']

# tsne_df.reset_index(drop = True, inplace = True)


# for i in range(len(tsne_df)):
#     if tsne_df.album_name[i] == 'PINES':
#         tsne_df.album_number[i] = 3

## TSNE Plot - Requested Artist

In [1781]:
## Resetting the html output to prevent multiple graphs from showing up

bpl.reset_output()

## Hover tool set to show album number, album name and song name

hover = bmo.HoverTool(
    tooltips = [
        ('Album #: ', '@album_number'),
        ('Album Name: ', '@album_name'),
        ('Song Name: ', '@song_name')])

## Configuring tools that will be used in the plot (plot size is large for viewing on monitor and not laptop)

p = figure(tools = [hover, "box_select, box_zoom, lasso_select, reset"], plot_width = 1200, plot_height = 800,  toolbar_location = 'above')

## Removing all lines and text, since location on the grid doesn't add any information for analysis

p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.xaxis.axis_line_color = None
p.xaxis.major_label_text_font_size = '0pt'

p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.yaxis.axis_line_color = None
p.yaxis.major_label_text_font_size = '0pt'

p.grid.visible = False

## Creating color palette/map

#palette = d3['Category20c']
color_map = bmo.CategoricalColorMapper(factors = tsne_df['album_name'].unique(), palette = palette_2) 

p.background_fill_color = '#F8F9FB'
p.background_fill_alpha = 0.0
p.outline_line_alpha = 0.0

## Creating plots and legend information

plot_names = []
legend_list = []

for i in range(len(album_name_list)):
    plot_names.append("tsne_plot_" + str(i))
     
for j in range(len(plot_names)):
    plot_names[j] = p.circle(x = 'x', y = 'y', source = tsne_df[tsne_df.album_name == album_name_list[j]], color = palette_2[j], 
                        size = 6, alpha = 0.95, muted_color = palette_2[j], muted_alpha = 0.1)
    legend_list.append((album_name_list[j], [plot_names[j]]))
    
legend = Legend(items = legend_list, location = (0,30))

p.add_layout(legend, 'right')
p.legend.click_policy = 'mute'
p.legend.border_line_alpha = 0.0
p.legend.background_fill_alpha = 0.0
p.legend.label_text_color = "black"
p.border_fill_alpha = 0.0

## Show the Results

#output_file("Arctic_Monkeys_TSNE_plot.html")
#bpl.output_notebook()
show(p)

#script, div = components(plot)
js, tag = bem.autoload_static(p, CDN, "/Users/justin/Code/A Fine Frenzy Webpage/Squadfree/aff_scripts/ff_script.js")


In [1785]:
#print(js)

In [1786]:
#print(tag)

## Predicting future album features

In [1096]:
## Creating a key:value dictionary pair from the unique features
## This will be used by spotify to recommend similar songs to our predicted values

keys = []
values = []

unique_feat_list = list(melted_df.feature.unique())

for i in unique_feat_list:
    keys.append(i)
    values.append(future_value(melted_df, i))

pred_dicts = dict(zip(keys, values))

## Spotify asks for the recommended song length in ms

pred_dicts['song_length_seconds'] = round(pred_dicts['song_length_seconds']*1000)

## Uses the artist seed and predicted values to return songs that are similiar

results = sp.recommendations(seed_artists = artist_search_id, limit = 10, min_popularity = 70,
                                     target_acousticness = pred_dicts['acousticness'],
                                     target_danceability = pred_dicts['danceability'],
                                     target_energy = pred_dicts['energy'],
                                     target_loudness = pred_dicts['loudness'],
                                     target_target_duration_ms = pred_dicts['song_length_seconds'],
                                     target_valence = pred_dicts['valence'])

## Unpack values and create a dataframe

results = results['tracks']

recommend_df = pd.DataFrame(results)

## Taking the track ID for additional analysis, plus the song popularity to incorporate popular areas to move into

recommended_tracks = []
popularity = []

for i in range(len(recommend_df)):
    recommended_tracks.append(recommend_df.uri[i])
    popularity.append(recommend_df.popularity[i])

## Call function and store audio features from recommended songs into a dataframe

recommend_audio_feat = sp_features_audio(recommended_tracks)

## Call function and store audio track features from recommended songs into a dataframe

recommend_track_analysis = audio_track_analysis(recommended_tracks)

## Creating counts for each track_analysis record

bars_count_rec = count_stats(0, recommend_track_analysis)
beats_count_rec = count_stats(1, recommend_track_analysis)
sections_count_rec = count_stats(2, recommend_track_analysis)
segments_count_rec = count_stats(3, recommend_track_analysis)
tatums_count_rec = count_stats(4, recommend_track_analysis)

audio_df_recommend = pd.DataFrame({'bars_count': bars_count, 'beats_count': beats_count, 
                      'sections_count': sections_count, 'segments_count': segments_count, 
                      'tatums_count': tatums_count})


## Calculating beats per minute and 

#song_len_rec = pd.Series(song_length_rec)

song_len_rec = recommend_audio_feat['duration_ms']

song_len_rec = song_len_rec.rename('song_length_seconds')

audio_analysis_rec = pd.concat([audio_df_recommend, song_len_rec], axis = 1)


# audio_analysis_rec = recommend_audio_feat

audio_analysis_rec['bpm'] = (audio_analysis_rec['beats_count'] / (audio_analysis_rec['song_length_seconds'] / 60))

## Merge track analysis and audio features, then append an album 

raw_df_recommend = pd.merge(recommend_audio_feat, audio_analysis_rec, how = 'left', left_index = True, right_index = True)
raw_df_recommend['album_number'] = len(raw_features_df.album_number.unique()) + 1

raw_df_recommend['spotify_link'] = 'https://embed.spotify.com/?uri=spotify:track:' + raw_df_recommend.id

## Popular Albums in the Same Space

In [1798]:
## Uses the artist seed and predicted values to return songs that are similiar

results = sp.recommendations(seed_artists = artist_search_id, limit = 10, min_popularity = 55,
                                     target_acousticness = pred_dicts['acousticness'],
                                     target_danceability = pred_dicts['danceability'],
                                     target_energy = pred_dicts['energy'],
                                     target_loudness = pred_dicts['loudness'],
                                     target_target_duration_ms = pred_dicts['song_length_seconds'],
                                     target_valence = pred_dicts['valence'])

## Unpack values and create a dataframe

results = results['tracks']

recommend_df = pd.DataFrame(results)

## Taking the track ID for additional analysis, plus the song popularity to incorporate popular areas to move into

recommended_tracks = []
popularity = []

for i in range(len(recommend_df)):
    recommended_tracks.append(recommend_df.uri[i])
    popularity.append(recommend_df.popularity[i])
  

  
## Creating a popularity Series for plotting

popularity_series = pd.Series(popularity)
popularity_series = popularity_series.rename("popularity")

In [1800]:
##AFF Only

# for i in range(len(recommend_df)):
#     print("https://open.spotify.com/track/" + recommend_df.id[i])

# recommended_tracks = recommended_tracks[1:]

# recommended_tracks.remove( 'spotify:track:71ehTADpxs85ULrZgSEKCy')

# recommended_tracks

# recommend_df = recommend_df.iloc[1:,]

# recommend_df = recommend_df.drop([6], axis = 0)

# recommend_df.reset_index(drop = True, inplace = True)

## Pulling data for recommendations for plotting

In [1862]:
## Audio features for each song

for i in range(len(recommended_tracks)):
    if i == 0:
        rec_audio_features_df = sp_features_audio(recommended_tracks[i])
    else:
        rec_new_audio_features_df = sp_features_audio(recommended_tracks[i])
        rec_audio_features_df = rec_audio_features_df.append(rec_new_audio_features_df)
        rec_audio_features_df.reset_index(drop = True, inplace = True)  

In [1863]:
## Track analysis for each song

rec_track_analysis_df = audio_track_analysis(recommended_tracks)
rec_track_analysis_df.drop(columns = ['meta', 'track'], inplace = True)

## Creating counts for each track_analysis record

bars_count = count_stats(0, rec_track_analysis_df)
beats_count = count_stats(1, rec_track_analysis_df)
sections_count = count_stats(2, rec_track_analysis_df)
segments_count = count_stats(3, rec_track_analysis_df)
tatums_count = count_stats(4, rec_track_analysis_df)

rec_audio_df = pd.DataFrame({'bars_count': bars_count, 'beats_count': beats_count, 
                      'sections_count': sections_count, 'segments_count': segments_count, 
                      'tatums_count': tatums_count})

In [1864]:
## Combining audio features and audio analysis features

rec_combined_df = pd.merge(rec_audio_df, rec_audio_features_df, how = 'left', left_index = True, right_index = True)

In [1865]:
## Calulating song length seconds

rec_combined_df['song_length_seconds'] = (rec_combined_df['duration_ms'] / 1000) 

## Calculating beats per minute

rec_combined_df['bpm'] = (rec_combined_df['beats_count'] / (rec_combined_df['song_length_seconds'] / 60))

In [1866]:
rec_combined_df_2 = rec_combined_df

In [1867]:
## Pulling song titles for recommended titles

rec_song_titles = sp_track_info(recommended_tracks)

rec_song_titles = rec_song_titles['name']

## Normalizing - Recommended Tracks

In [1868]:
## Normalizing data for plotting

rec_feature_col_names = ['acousticness',
                     'danceability',
                     'energy',
                     'loudness',
                     'bars_count',
                     'beats_count',
                     'sections_count',
                     'segments_count',
                     'tatums_count',
                     'song_length_seconds',
                     'bpm']


In [1869]:
## Normalizing data for plotting

feature_col_names = ['acousticness',
                     'danceability',
                     'energy',
                     'loudness',
                     'bars_count',
                     'beats_count',
                     'sections_count',
                     'segments_count',
                     'tatums_count',
                     'song_length_seconds',
                     'bpm']


In [1870]:
rec_combined_df = pd.concat([rec_combined_df, rec_song_titles], axis = 1)

rec_combined_df = rec_combined_df.rename(columns = {'name': 'song_name'})

## Adding "album_number" to match what I used in the original fitting
rec_combined_df['album_number'] = len(raw_features_df_2.album_number.unique()) + 1

In [1871]:
raw_features_df_2.drop(columns = ['id'], inplace = True)

ValueError: labels ['id'] not contained in axis

In [1872]:
combined_normalization = pd.concat([rec_combined_df,raw_features_df_2], axis = 0)

## Removing duplicates that may have come from the recommendations

combined_normalization = combined_normalization[combined_normalization.duplicated(subset = 'song_name', keep = 'last') == False]
combined_normalization.reset_index(drop = True, inplace = True)


In [1873]:
rec_song_list_unique = combined_normalization[pd.isnull(combined_normalization.album_name)]

rec_song_list_unique = rec_song_list_unique['song_name']

In [1874]:
## Normalizing data for plotting

feature_col_names = ['acousticness',
                     'danceability',
                     'energy',
                     'loudness',
                     'bars_count',
                     'beats_count',
                     'sections_count',
                     'segments_count',
                     'tatums_count',
                     'song_length_seconds',
                     'bpm']

## Iterating through columns to normalize the data 

for i in feature_col_names:
    combined_normalization[i] = feature_norm(combined_normalization, i)

In [1875]:
combined_normalization = combined_normalization.sort_values(by = ['album_number', 'track_number'], ascending = True)

## PCA - Recommended tracks

In [1876]:
## Reducing features

rec_pca_df = combined_normalization[[
                     'album_number',
                     'acousticness',
                     'danceability',
                     'energy',
                     'bars_count',
                     'sections_count',
                     'segments_count',
                     'tatums_count',
                     'song_length_seconds',
                     'bpm']]


## Splitting data into target and features

rec_y = rec_pca_df['album_number']
rec_X_norm = rec_pca_df.iloc[:,1:]

## Transforming using previously-fit PCA model

rec_transformed = pd.DataFrame(pca.transform(rec_X_norm))

## Print statements to show the impact of each feature

pca_len = len(pca.explained_variance_)

## LDA & TSNE - Recommended Tracks

In [1877]:
## Transforming using previously-fit LDA model

rec_transformed_lda = pd.DataFrame(lda.transform(rec_X_norm))

## Defining 4 components for LDA and applying fit_transform

tsne_req = TSNE(n_components = 2, perplexity = 60, learning_rate = 900, n_iter = 1000)

rec_transformed_tsne = pd.DataFrame(tsne.fit_transform(rec_transformed_lda))

## Creating new df for interactive Bokeh graphs

rec_tsne_df = pd.concat([rec_transformed_tsne.iloc[:,:2]], axis=1, join_axes=[rec_combined_df.index])

rec_tsne_df.rename(columns = {0 : 'x', 1 : 'y'}, inplace = True)

# ## Appending popularity

# rec_tsne_df = pd.concat([rec_tsne_df, popularity_series], axis = 1)

## Appending song name

rec_tsne_df = pd.concat([rec_tsne_df, rec_song_list_unique], axis = 1)

## Creating album name for recommended songs

rec_tsne_df['album_name'] = 'Recommended Popular Song'

# Creating album name for recommended songs

rec_tsne_df['album_number'] = 'None'

rec_tsne_df = rec_tsne_df[pd.notnull(rec_tsne_df.song_name)]

In [1878]:
## Pulling out the original tracks - now normalized again

original_tracks_tojoin = rec_transformed_tsne.iloc[len(rec_tsne_df):,]

original_tracks_tojoin.reset_index(drop = True, inplace = True)

original_tracks_tojoin.rename(columns = {0 : 'x', 1 : 'y'}, inplace = True)

original_tracks_tsne_df = pd.concat([original_tracks_tojoin, raw_features_df_2[['song_name', 'album_name', 'album_number']]], axis=1, join_axes=[raw_features_df_2.index])

/Users/justin/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [1879]:
overlay_tsne_df = original_tracks_tsne_df.append(rec_tsne_df)

overlay_album_list = list(overlay_tsne_df.album_name.unique())

In [1880]:
overlay_tsne_df = overlay_tsne_df[overlay_tsne_df.duplicated(subset = ['album_name', 'song_name'], keep = 'first') == False]

In [1881]:
overlay_tsne_df.reset_index(drop = True, inplace = True)

for i in range(len(tsne_df)):
    if tsne_df.album_name[i] == 'PINES':
        tsne_df.album_number[i] = 3

/Users/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [1882]:
# overlay_album_list  = ['Please Please Me (Remastered)', 'With The Beatles (Remastered)', "A Hard Day's Night (Remastered)",
#                        'Beatles For Sale (Remastered)', 'Help! (Remastered)', 'Rubber Soul (Remastered)', 'Revolver (Remastered)',
#                        "Sgt. Pepper's Lonely Hearts Club Band (Remastered)", 'Magical Mystery Tour (Remastered)',
#                        'The Beatles (Remastered)', 'Yellow Submarine (Remastered)', 'Abbey Road (Remastered)', 
#                        'Let It Be (Remastered)', 'Recommended Popular Song']

In [1883]:
## AFF Only

# palette_2 = ['#000000', '#525b91', '#4e6b4e']

## Plotting TSNE with Recommended Tracks overlay

In [1884]:
## Resetting the html output to prevent multiple graphs from showing up

bpl.reset_output()

## Hover tool set to show album number, album name and song name

hover = bmo.HoverTool(
    tooltips = [
        ('Album #: ', '@album_number'),
        ('Album Name: ', '@album_name'),
        ('Song Name: ', '@song_name')])

## Configuring tools that will be used in the plot (plot size is large for viewing on monitor and not laptop)

p = figure(tools = [hover, "box_select, box_zoom, lasso_select, reset"], plot_width = 1200, plot_height = 800,  toolbar_location = 'above')

## Removing all lines and text, since location on the grid doesn't add any information for analysis

p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.xaxis.axis_line_color = None
p.xaxis.major_label_text_font_size = '0pt'

p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.yaxis.axis_line_color = None
p.yaxis.major_label_text_font_size = '0pt'

p.grid.visible = False

## Creating color palette/map
palette_3 = ['#bc1818']
# palette_3 = ['#1E1E1E']
# palette = d3['Category20c'][len(overlay_tsne_df['album_name'].unique())-1]
# palette = palette
palette_2.extend(palette_3)
color_map = bmo.CategoricalColorMapper(factors = overlay_tsne_df['album_name'].unique(), palette = palette_2) 

p.background_fill_color = '#F8F9FB'
p.background_fill_alpha = 0.0
p.outline_line_alpha = 0.0

## Creating plots and legend information

plot_names = []
legend_list = []

for i in range(len(overlay_album_list)):
    plot_names.append("tsne_plot_" + str(i))
     
for j in range(len(plot_names)):
    plot_names[j] = p.circle(x = 'x', y = 'y', source = overlay_tsne_df[overlay_tsne_df.album_name == overlay_album_list[j]], color = palette_2[j], 
                        size = 6, alpha = 0.75, muted_color = palette_2[j], muted_alpha = 0.1)
    legend_list.append((overlay_album_list[j], [plot_names[j]]))
    
legend = Legend(items = legend_list, location = (0,30))

p.add_layout(legend, 'right')
p.legend.click_policy = 'mute'
p.legend.border_line_alpha = 0.0
p.legend.background_fill_alpha = 0.0
p.legend.label_text_color = "black"
p.border_fill_alpha = 0.0

## Show the Results

# output_file("Arctic_Monkeys_song_TSNE_plot.html")
# bpl.output_notebook()
show(p)

#script, div = components(plot)
js, tag = bem.autoload_static(p, CDN, "/Users/justin/Code/A Fine Frenzy Webpage/Squadfree/aff_scripts/aff_rec_script.js")

In [1910]:
#print(js)

In [1911]:
#print(tag)

## Creating album feature plots

In [1887]:
## Calculating means for each of the original albums

combined_norm_melt_df = combined_normalization[['album_number', 'album_name',
        'bars_count', 'danceability', 'energy', 'acousticness']]

original_album_means = combined_norm_melt_df.groupby(['album_number', 'album_name']).mean().reset_index()

In [1888]:
original_album_means = original_album_means[original_album_means.duplicated(subset = 'album_name', keep = 'first') == False]
original_album_means.reset_index(drop = True, inplace = True)

In [1890]:
## Renaming with weird spacing for plotting purposes

original_album_means = original_album_means.rename(columns = {'album_name' : 'Album Name',
                                                              'acousticness' : 'Acousticness', 
                                                              'danceability' : 'Danceability', 
                                                              'energy' : 'Energy         ', 
                                                              'bars_count' : '                Bars Count   ', 
                                                              #'sections_count' : '                         Sections Count'
                                                             })

In [1891]:
## Calculating means for each of the recommended tracks

rec_song_means = combined_norm_melt_df[pd.isnull(combined_norm_melt_df.album_name)]

rec_song_means = rec_song_means.drop(columns = ['album_number', 'album_name'])

rec_song_means = pd.concat([rec_song_means, rec_song_list_unique], axis=1, join_axes=[rec_song_titles.index])

In [1892]:
rec_song_means = rec_song_means[pd.notnull(rec_song_means.song_name)]

In [1893]:
rec_song_means = rec_song_means.groupby(['song_name']).mean().reset_index()

In [1902]:
## Renaming with weird spacing for plotting purposes

rec_song_means = rec_song_means.rename(columns = {'song_name' : 'Song Name',
                                                              'acousticness' : 'Acousticness', 
                                                              'danceability' : 'Danceability', 
                                                              'energy' : 'Energy         ', 
                                                              'bars_count' : '                Bars Count   ', 
                                                              #'sections_count' : '                         Sections Count'
                                                             })

In [1903]:
rec_song_means = rec_song_means[pd.notnull(rec_song_means.iloc[:,1])]

rec_song_means.reset_index(drop = True, inplace = True)

In [1904]:
original_album_means.to_csv('original_album_means.csv', index = False)
#original_album_means = pd.read_csv('original_album_means.csv')

rec_song_means.to_csv('rec_song_means.csv', index = False)
#rec_song_means = pd.read_csv('rec_song_means.csv')

## Creating radar plots using matplotlib

In [1905]:
def create_radar (df, col):
    
    ## Create list of 'categories' for chart

    categories = list(df)[2:]
    cat_len = len(categories)

    ## Flatten df to plot the first record

    plot_vals = df.loc[col].drop(['album_number', 'Album Name']).values.flatten().tolist()

    plot_vals += plot_vals[:1]

    plot_vals 

    ## Calculating angle for plot (code found online - math stuff...)

    angles = [n / float(cat_len) * 2 * pi for n in range(cat_len)]
    angles += angles[:1]

    ## Creating the plot - single plot

    ax = plt.subplot(111, polar=True)

    plt.xticks(angles[:-1], categories, color='grey', size=15)

    ## Not displaying values, since all values are normalized and doesn't enhance user experience

    plt.yticks([0,0.5,1.0], [""], color="grey", size=10)

    ## All normalized values are between 0 and 1
    plt.ylim(0,1.0)
    
    # Plot data
    ax.plot(angles, plot_vals, linewidth=2, linestyle='solid', color = '#92b597')

    # Fill area
    ax.fill(angles, plot_vals, '#92b597', alpha=0.6)

In [1906]:
## Creating and saving a radar chart for each album, then exporting results

original_radar_names = []

for i in range(len(original_album_means)):
    radar_name = original_album_means['Album Name'][i]
    original_radar_names.append(radar_name)
    original_radar_names[i] = create_radar(original_album_means, i)
    save_location = 'radar_charts/' + radar_name + '.png'
    plt.savefig(save_location, dpi = 800, transparent = True)
    plt.gcf().clear()

In [1907]:
def create_rec_radar (df, col):
    
    ## Create list of 'categories' for chart

    categories = list(df)[1:]
    cat_len = len(categories)

    ## Flatten df to plot the first record

    plot_vals = df.loc[col].drop(['Song Name']).values.flatten().tolist()

    plot_vals += plot_vals[:1]

    plot_vals

    ## Calculating angle for plot (code found online - math stuff...)

    angles = [n / float(cat_len) * 2 * pi for n in range(cat_len)]
    angles += angles[:1]

    ## Creating the plot - single plot

    ax = plt.subplot(111, polar=True)

    plt.xticks(angles[:-1], categories, color='grey', size=15)

    ## Not displaying values, since all values are normalized and doesn't enhance user experience

    plt.yticks([0,0.5,1.0], [""], color="grey", size=10)

    ## All normalized values are between 0 and 1
    plt.ylim(0,1.0)

    # Plot data
    ax.plot(angles, plot_vals, linewidth=1, linestyle='solid', color = 'r')

    # Fill area
    ax.fill(angles, plot_vals, 'r', alpha=0.2)

In [1909]:
## Creating and saving a radar chart for each album, then exporting results

rec_radar_names = []

for i in range(len(rec_song_means)):
    radar_names = rec_song_means['Song Name'][i] + "_Recommended"
    rec_radar_names.append(radar_names)
    rec_radar_names[i] = create_rec_radar(rec_song_means, i)
    save_location = 'radar_charts/' + radar_names + '.png'
    plt.savefig(save_location, dpi = 800, transparent = True)
    plt.gcf().clear()

## Scrap Work Below

In [909]:
print(time.time() - start_time)

411.3618731498718
